In [2]:
import dowhy
import pandas as pd

In [3]:
df = pd.read_csv('data/all_types.csv')

In [4]:
df.head()

,Age,Gender,Race,Length of Hospital Stay (LOS),Height_cm,Weight-kg,BMI,sPESI,PESI Class,PE Severity 1=low risk; 2=intermediate; 3=massive; 4=indeterminate,...,Platelet <70k,Etiology of Clot,adverseEvent,type,Required supplemental O2,"Acute DVT (deep venous thrombosis), 3 = chronic",Elevated Initial Troponin,RV/LV Ratio .85 to .90,RV/LV Ratio .91-.94,RV/LV Ratio .95 or greater
0,72,M,W,3,175.3,83.5,27.2,0,2,1,...,0.0,trauma,0,saddle,NaN,NaN,NaN,NaN,NaN,NaN
1,74,F,W,3,170.2,76.3,26.3,0,2,2,...,0.0,surgery,0,saddle,NaN,NaN,NaN,NaN,NaN,NaN
2,57,F,B,3,152.4,97.1,41.8,0,2,1,...,0.0,trauma,0,saddle,NaN,NaN,NaN,NaN,NaN,NaN
3,64,M,W,3,175.3,72.2,23.5,0,2,2,...,0.0,immobility- car travel,0,saddle,NaN,NaN,NaN,NaN,NaN,NaN
4,68,M,B,8,152.4,80.3,34.6,1,5,2,...,0.0,personal history,0,saddle,NaN,NaN,NaN,NaN,NaN,NaN
